# Assignment 1

In [1]:
# Import Libraries
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import register_matplotlib_converters
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
from lifelines import KaplanMeierFitter
register_matplotlib_converters()
sns.set(style="whitegrid")

In [2]:
# Load Client Information dataset
def load_data(file_path, date_cols):
    return pd.read_csv(
        file_path,
        sep=';',
        encoding='utf-8-sig',
        parse_dates=date_cols,
        dayfirst=True
    )

client_df = load_data('Client_information.csv', ['Registration_date'])
trans_df = load_data('Transactions.csv', ['Date'])

/scratch-node/20230112.2631922/ipykernel_1385445/2424773142.py:3: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.read_csv(
/scratch-node/20230112.2631922/ipykernel_1385445/2424773142.py:3: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.read_csv(


## General data source description

In [3]:
trans_df.columns

Index(['City', 'Date', 'Receipt_id', 'Sale_Return', 'Client_id', 'Channel',
       'Platform', 'Credit', 'SKU_id', 'Business', 'Brand', 'Category',
       'Region', 'Revenue', 'Quantity', 'City_On_Off', 'Category_Ukr'],
      dtype='object')

In [4]:
client_df.columns

Index(['Client_id', 'Registration_date', 'Gender', 'Age', 'Number_of_receipts',
       'Total_revenue'],
      dtype='object')

In [5]:
client_df

,Client_id,Registration_date,Gender,Age,Number_of_receipts,Total_revenue
0,509785107,2019-11-30,-,41,68,3067388
1,967337576,2018-07-19,-,36,234,3002606
2,675309125,2014-01-25,-,63,154,2822662
3,663403250,2023-07-13,-,22,78,2556307
4,675931029,2016-05-31,male,40,333,2407277
...,...,...,...,...,...,...
411126,506485479,2024-05-07,-,56,1,0
411127,950131224,2024-11-30,-,22,1,0
411128,960627253,2024-08-29,-,,1,0
411129,990001994,2023-11-14,-,33,1,0


In [6]:
trans_df

,City,Date,Receipt_id,Sale_Return,Client_id,Channel,Platform,Credit,SKU_id,Business,Brand,Category,Region,Revenue,Quantity,City_On_Off,Category_Ukr
0,Kam'yanske,2023-05-22,22.05.2023-ЧDNZ7-0001,1,674537053,Retail,,0,2194716,Accessories,Philips,Headphones,Dnipropetrovska,1098,1,Off+ON,Навушники
1,Kam'yanske,2024-04-13,13.04.2024-Ч002003-0003,1,679873829,Retail,,0,1603317,Large household appliances,Samsung,Air conditioner,Dnipropetrovska,18299,1,Off+ON,Кондиціонер
2,Kam'yanske,2024-05-19,19.05.2024-Ч002002-0041,1,975843714,Retail,,0,1016249,Accessories,Slon,Accessories for traditional vacuum cleaners,Dnipropetrovska,318,2,Off+ON,Аксесуари для традиційного пилососа
3,Kam'yanske,2024-05-16,16.05.2024-Ч002001-0079,1,973446093,Retail,,0,2268919,Entertainment and Computer equipment,Lenovo,Tablet computer,Dnipropetrovska,11998,1,Off+ON,Планшетний комп'ютер
4,Kam'yanske,2023-05-27,27.05.2023-ЧDNZ2-0131,1,671186761,Retail,,0,1786301,COMFY HOME,Finish,Dishwasher tablet,Dnipropetrovska,287,1,Off+ON,Таблетка для посудомийної машини
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2679911,Ohlyadiv,2025-03-12,12.03.2025-Ч029308-0892,1,971409475,Internet,app,0,2479673,Small household appliances,Saturn,Meat grinder,Lvivska,1599,1,ON,М'ясорубка
2679912,Chornoparivka,2025-03-12,12.03.2025-Ч029308-0908,1,672332195,Internet,web,0,1816457,Innovations and Telecom,MOBVOI,Smart watch and fitness bracelet,Dnipropetrovska,4999,1,ON,Смарт-годинник та фітнес-браслет
2679913,Chornoparivka,2025-03-12,12.03.2025-Ч029308-0908,1,672332195,Internet,web,0,7139,Promotional services,Comfy,Paid delivery,Dnipropetrovska,55,1,ON,Доставка платна
2679914,Pidryasne,2025-03-11,11.03.2025-Ч029618-0004,1,502237932,Internet,web,0,7139,Promotional services,Comfy,Paid delivery,Lvivska,1,1,ON,Доставка платна


In [7]:
def classify_columns(df):
    numerical = df.select_dtypes(include=[np.number]).columns.tolist()
    categorical = df.select_dtypes(include=['object']).columns.tolist()
    datetime_cols = df.select_dtypes(include=['datetime']).columns.tolist()
    
    return {
        'numerical': numerical,
        'categorical': categorical,
        'datetime': datetime_cols
    }

client_info = classify_columns(client_df)
print("Client Information Dataset:")
print(f"Numerical columns: {client_info['numerical']}")
print(f"Categorical columns: {client_info['categorical']}")
print(f"Datetime columns: {client_info['datetime']}")
transactions = classify_columns(trans_df)
print("\nTransactions Dataset:")
print(f"Numerical columns: {transactions['numerical']}")
print(f"Categorical columns: {transactions['categorical']}")
print(f"Datetime columns: {transactions['datetime']}")

Client Information Dataset:
Numerical columns: ['Client_id', 'Number_of_receipts', 'Total_revenue']
Categorical columns: ['Gender', 'Age']
Datetime columns: ['Registration_date']

Transactions Dataset:
Numerical columns: ['Sale_Return', 'Client_id', 'Credit', 'SKU_id']
Categorical columns: ['City', 'Receipt_id', 'Channel', 'Platform', 'Business', 'Brand', 'Category', 'Region', 'Revenue', 'Quantity', 'City_On_Off', 'Category_Ukr']
Datetime columns: ['Date']


In [8]:
def preprocess_transactions(df):
    # Convert Revenue to numeric
    if 'Revenue' in df.columns:
        df['Revenue'] = df['Revenue'].astype(str).str.replace(
            '[\$,]', '', regex=True
        ).replace('nan', np.nan)
        
        df['Revenue'] = pd.to_numeric(df['Revenue'], errors='coerce')
    
    if 'Quantity' in df.columns:
        df['Quantity'] = pd.to_numeric(
            df['Quantity'].astype(str).str.replace(',', ''),
            errors='coerce'
        )
    
    return df

trans_df = preprocess_transactions(trans_df)

<>:5: DeprecationWarning: invalid escape sequence '\$'
<>:5: DeprecationWarning: invalid escape sequence '\$'
/scratch-node/20230112.2631922/ipykernel_1385445/2746078548.py:5: DeprecationWarning: invalid escape sequence '\$'
  '[\$,]', '', regex=True


In [9]:
def simple_duplicate_check(client_df, trans_df):
    
    client_dupes = client_df[client_df.duplicated('Client_id', keep=False)]
    
    # Transaction duplicates (match on receipt + date + amount)
    trans_dupes = trans_df[trans_df.duplicated(
        subset=['Receipt_id', 'Date', 'Revenue'], 
        keep=False
    )]
    print(f"Clients with duplicate IDs: {len(client_dupes)}")
    print(f"Transactions with duplicate receipts: {len(trans_dupes)}")
    
    return client_dupes, trans_dupes

In [10]:
client_dupes, trans_dupes = simple_duplicate_check(client_df, trans_df)

Clients with duplicate IDs: 0
Transactions with duplicate receipts: 62261


In [11]:
def analyze_dataset(df, dataset_name):
    
    for col in df.columns:
        print(f"\n{'-'*50}")
        print(f"Column: {col}")
        print(f"Data Type: {df[col].dtype}")
        
        missing = df[col].isnull().sum()
        missing_pct = round(missing / len(df) * 100, 2)
        print(f"Missing Values: {missing} ({missing_pct}%)")
        
        if pd.api.types.is_numeric_dtype(df[col]):
            print("\nNumeric Analysis:")
            stats = df[col].describe()
            
            print(f"  Min: {stats['min']:.2f}")
            print(f"  Q1: {stats['25%']:.2f}")
            print(f"  Median: {stats['50%']:.2f}")
            print(f"  Q3: {stats['75%']:.2f}")
            print(f"  Max: {stats['max']:.2f}")
            
            q1 = stats['25%']
            q3 = stats['75%']
            iqr = q3 - q1
            lower_bound = q1 - 1.5 * iqr
            upper_bound = q3 + 1.5 * iqr
            outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
            outlier_pct = round(len(outliers)/len(df)*100, 2)
            print(f"  Outliers: {len(outliers)} ({outlier_pct}%)")
            print(f"  Outlier Interval: [{lower_bound:.2f}, {upper_bound:.2f}]")
            
            neg_mask = df[col] < 0
            neg_count = neg_mask.sum()
            if neg_count > 0:
                neg_pct = round(neg_count/len(df)*100, 2)
                print(f"  Negative Values: {neg_count} ({neg_pct}%)")
                print(f"  Sample Negatives: {df[col][neg_mask].head(3).tolist()}...")
            else:
                print("  No negative values detected")
        
        elif pd.api.types.is_categorical_dtype(df[col]) or pd.api.types.is_object_dtype(df[col]):
            print("\nCategorical Analysis:")
            unique_vals = df[col].unique()
            unique_count = len(unique_vals)
            print(f"  Unique Values: {unique_count}")
            
            print("\n  Value Description:")
            if unique_count <= 10:
                print(f"  All Values: {unique_vals}")
            else:
                print(f"  First 10 Values: {unique_vals[:10]}")
                print(f" ({unique_count-10} more values)")
            
            print("\n  Frequency Table (Top 10):")
            freq = df[col].value_counts(normalize=True, dropna=False).head(10)
            for value, pct in freq.items():
                count = df[col].value_counts(dropna=False).loc[value]
                print(f"  {value}: {count} ({pct*100:.2f}%)")
            
            if df[col].isnull().any():
                nan_pct = round(df[col].isnull().mean()*100, 2)
                print(f"\n  Missing Category: {nan_pct}%")
        
        elif pd.api.types.is_datetime64_any_dtype(df[col]):
            print("\nDate Analysis:")
            print(f"  Earliest Date: {df[col].min()}")
            print(f"  Latest Date: {df[col].max()}")
            print(f"  Date Range: {df[col].max() - df[col].min()}")

In [12]:
# Analyze Client Information Dataset
print("Analyzing Client Information Dataset")
analyze_dataset(client_df, "Client Information")

Analyzing Client Information Dataset

--------------------------------------------------
Column: Client_id
Data Type: int64
Missing Values: 0 (0.0%)

Numeric Analysis:
  Min: 335026407.00
  Q1: 667611028.00
  Median: 933415200.00
  Q3: 972299549.50
  Max: 999880597.00
  Outliers: 0 (0.0%)
  Outlier Interval: [210578245.75, 1429332331.75]
  No negative values detected

--------------------------------------------------
Column: Registration_date
Data Type: datetime64[ns]
Missing Values: 0 (0.0%)

Date Analysis:
  Earliest Date: 2011-10-21 00:00:00
  Latest Date: 2025-03-11 00:00:00
  Date Range: 4890 days 00:00:00

--------------------------------------------------
Column: Gender
Data Type: object
Missing Values: 0 (0.0%)

Categorical Analysis:
  Unique Values: 3

  Value Description:
  All Values: ['-' 'male' 'female']

  Frequency Table (Top 10):
  -: 386369 (93.98%)
  female: 13073 (3.18%)
  male: 11689 (2.84%)

--------------------------------------------------
Column: Age
Data Type:

In [13]:
print("\n\nAnalyzing Transactions Dataset")
analyze_dataset(trans_df, "Transactions")



Analyzing Transactions Dataset

--------------------------------------------------
Column: City
Data Type: object
Missing Values: 0 (0.0%)

Categorical Analysis:
  Unique Values: 3786

  Value Description:
  First 10 Values: ["Kam'yanske" 'Velyka Dymerka' 'Murovane' 'Zaporizhzhya' 'Kropyvnytskyy'
 'Dnipro' 'Pokrovsk' 'Kremenchuk' 'Kryvyy Rih' 'Mykolayiv']
 (3776 more values)

  Frequency Table (Top 10):
  Kyyiv: 425763 (15.89%)
  Dnipro: 270417 (10.09%)
  Kharkiv: 172802 (6.45%)
  Odesa: 134593 (5.02%)
  Kryvyy Rih: 130396 (4.87%)
  Lviv: 119090 (4.44%)
  Zaporizhzhya: 100410 (3.75%)
  Sumy: 58505 (2.18%)
  Cherkasy: 57932 (2.16%)
  Mykolayiv: 57711 (2.15%)

--------------------------------------------------
Column: Date
Data Type: datetime64[ns]
Missing Values: 0 (0.0%)

Date Analysis:
  Earliest Date: 2023-01-01 00:00:00
  Latest Date: 2025-03-12 00:00:00
  Date Range: 801 days 00:00:00

--------------------------------------------------
Column: Receipt_id
Data Type: object
Missing

In [14]:
def clean_transaction_data(trans_df, client_df):
    trans_df['Revenue'] = pd.to_numeric(
        trans_df['Revenue'].astype(str).str.replace('[\$,]', '', regex=True),
        errors='coerce'
    )
    trans_df['Quantity'] = pd.to_numeric(
        trans_df['Quantity'].astype(str).str.replace(',', ''),
        errors='coerce'
    )
    trans_df['Sale_Return'] = trans_df['Sale_Return'].astype(int)
    valid_mask = (
        ((trans_df['Sale_Return'] == 1) & 
         (trans_df['Revenue'] > 0) & 
         (trans_df['Quantity'] > 0)) |
        ((trans_df['Sale_Return'] == 0) & 
         (trans_df['Revenue'] < 0) & 
         (trans_df['Quantity'] < 0))
    )
    cleaned_df = trans_df[valid_mask & 
                         trans_df['Client_id'].isin(client_df['Client_id'])].copy()
    sales_df = cleaned_df[cleaned_df['Sale_Return'] == 1]
    returns_df = cleaned_df[cleaned_df['Sale_Return'] == 0]
    assert sales_df[['Revenue', 'Quantity']].ge(0).all().all(), "Invalid sales data"
    assert returns_df[['Revenue', 'Quantity']].lt(0).all().all(), "Invalid returns data"
    
    return sales_df, returns_df
sales_df, returns_df = clean_transaction_data(trans_df, client_df)

<>:3: DeprecationWarning: invalid escape sequence '\$'
<>:3: DeprecationWarning: invalid escape sequence '\$'
/scratch-node/20230112.2631922/ipykernel_1385445/979911352.py:3: DeprecationWarning: invalid escape sequence '\$'
  trans_df['Revenue'].astype(str).str.replace('[\$,]', '', regex=True),


In [15]:
print(f"Sales transactions: {len(sales_df)}")
print(f"Return transactions: {len(returns_df)}")

Sales transactions: 2569803
Return transactions: 51166


In [16]:
def descriptive_stats(df):
    stats = {}
    for col in df.select_dtypes(include=[np.number]).columns:
        series = df[col]
        q1 = series.quantile(0.25)
        q3 = series.quantile(0.75)
        iqr = q3 - q1
        outlier_range = (q1 - 1.5*iqr, q3 + 1.5*iqr)
        outliers = series[(series < outlier_range[0]) | (series > outlier_range[1])]
        
        stats[col] = {
            'mean': round(series.mean(), 2),
            'std': round(series.std(), 2),
            'min': series.min(),
            '25%': q1,
            '50%': series.median(),
            '75%': q3,
            'max': series.max(),
            'outlier_range': (round(outlier_range[0], 2), round(outlier_range[1], 2)),
            'outlier_pct': round(len(outliers)/len(series)*100, 2)
        }
    return pd.DataFrame(stats).T

client_stats = descriptive_stats(client_df)
transaction_stats = descriptive_stats(trans_df)

In [17]:
client_stats

,mean,std,min,25%,50%,75%,max,outlier_range,outlier_pct
Client_id,809132478.62,175110863.07,335026407,667611028.0,933415200.0,972299549.5,999880597,"(210578245.75, 1429332331.75)",0.0
Number_of_receipts,7.64,11.19,1,2.0,4.0,9.0,454,"(-8.5, 19.5)",9.34
Total_revenue,39769.31,66273.14,0,4287.5,17339.0,48626.5,3067388,"(-62221.0, 115135.0)",8.16


In [18]:
transaction_stats

,mean,std,min,25%,50%,75%,max,outlier_range,outlier_pct
Sale_Return,0.98,0.14,0,1.0,1.0,1.0,1,"(1.0, 1.0)",1.97
Client_id,808401338.24,175588511.46,335026407,667312033.0,933304633.0,972236946.0,999880597,"(209924663.5, 1429624315.5)",0.0
Credit,0.16,0.37,0,0.0,0.0,0.0,1,"(0.0, 0.0)",16.44
SKU_id,1949150.59,800618.08,152,1538079.0,2023773.0,2552219.0,3397275,"(16869.0, 4073429.0)",7.18
Revenue,4439.6,422531.21,-288000.0,189.0,699.0,2899.0,539909999.0,"(-3876.0, 6964.0)",15.4
Quantity,1.0,0.47,-20.0,1.0,1.0,1.0,100.0,"(1.0, 1.0)",4.59


In [19]:
# Visualization functions
def plot_distribution(df, column, title):
    plt.figure(figsize=(12, 6))
    sns.histplot(df[column], kde=True, bins=30)
    plt.title(f'Distribution of {title}')
    plt.xlabel(column)
    plt.ylabel('Frequency')
    plt.show()

def plot_categorical(df, column, title):
    plt.figure(figsize=(12, 6))
    df[column].value_counts().plot(kind='bar')
    plt.title(f'Distribution of {title}')
    plt.xlabel(column)
    plt.ylabel('Count')
    plt.xticks(rotation=45)
    plt.show()

def plot_time_series(df, date_col, value_col, freq='M'):
    df_temp = df.set_index(date_col).resample(freq)[value_col].sum().reset_index()
    plt.figure(figsize=(14, 7))
    sns.lineplot(x=date_col, y=value_col, data=df_temp)
    plt.title(f'{value_col} Over Time ({freq})')
    plt.xticks(rotation=45)
    plt.show()

def plot_boxplot(df, column):
    plt.figure(figsize=(10, 6))
    sns.boxplot(x=df[column])
    plt.title(f'Boxplot of {column}')
    plt.show()

def plot_correlation_heatmap(df, title):
    corr_matrix = df.select_dtypes(include=[np.number]).corr()
    plt.figure(figsize=(12, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
    plt.title(title)
    plt.show()

In [20]:
plot_distribution(client_df, 'Age', 'Client Age Distribution')
plot_distribution(client_df, 'Total_revenue', 'Total Revenue Distribution')
plot_distribution(trans_df, 'Revenue', 'Transaction Revenue Distribution')
plot_boxplot(trans_df, 'Revenue')
plot_categorical(trans_df, 'Category', 'Transaction Categories')
plot_categorical(trans_df, 'Region', 'Transactions by Region')

In [21]:
plot_time_series(sales_df, 'Date', 'Revenue', 'M')
plot_time_series(sales_df, 'Date', 'Quantity', 'W')

In [22]:
print("\nClient Data Correlation Matrix:")
plot_correlation_heatmap(client_df, 'Client Data Correlation')
print("\nTransaction Data Correlation Matrix:")
plot_correlation_heatmap(trans_df, 'Transaction Data Correlation')


Client Data Correlation Matrix:

Transaction Data Correlation Matrix:


# Assignment 2

In [23]:
# RFM + KMeans Clustering for segmentation

In [24]:
snapshot_date = trans_df['Date'].max() + dt.timedelta(days=1)

rfm_df = trans_df.groupby('Client_id').agg({
    'Date': lambda x: (snapshot_date - x.max()).days,   
    'Receipt_id': 'nunique',                            
    'Revenue': 'sum'                                    
}).reset_index()

rfm_df.columns = ['Client_id', 'Recency', 'Frequency', 'Monetary']

scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm_df[['Recency', 'Frequency', 'Monetary']])

kmeans = KMeans(n_clusters=4, random_state=42)
rfm_df['Cluster'] = kmeans.fit_predict(rfm_scaled)

print(rfm_df.head())

   Client_id  Recency  Frequency  Monetary  Cluster
0  335026407      164          1    1276.0        1
1  372456105      305          1    7999.0        1
2  381523648      226          1    1429.0        1
3  442348055      204          1    6653.0        1
4  443000507      253          1    2999.0        1


In [25]:
print(f"Total clients: {rfm_df.shape[0]}")
print(rfm_df['Cluster'].value_counts())

Total clients: 411129
Cluster
0    230908
1    143455
2     36760
3         6
Name: count, dtype: int64


In [26]:
# may take more time to run depend on the cores [comfy can change min_support to 0.001 for more better results i don't have cores for that to test]

In [27]:
basket = trans_df.groupby('Receipt_id')['SKU_id'].apply(list)
basket_filtered = basket[basket.apply(len) > 1]

te = TransactionEncoder()
basket_encoded = te.fit(basket_filtered).transform(basket_filtered)
df_basket = pd.DataFrame(basket_encoded, columns=te.columns_)

frequent_itemsets = apriori(df_basket, min_support=0.01, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)
rules = rules.sort_values(by='lift', ascending=False)

print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(1000))

  antecedents consequents   support  confidence       lift
0   (2309558)   (1550702)  0.011599    0.820096  11.826354
1   (1550702)   (2309558)  0.011599    0.167267  11.826354


In [28]:
trans_df['is_return'] = trans_df['Revenue'] < 0
trans_df['Date'] = pd.to_datetime(trans_df['Date'])

purchases = trans_df[trans_df['Revenue'] >= 0]
returns = trans_df[trans_df['is_return']]

first_purchase = purchases.groupby(['Client_id', 'SKU_id'])['Date'].min().reset_index()
first_purchase.rename(columns={'Date': 'purchase_date'}, inplace=True)

first_return = returns.groupby(['Client_id', 'SKU_id'])['Date'].min().reset_index()
first_return.rename(columns={'Date': 'return_date'}, inplace=True)
sale_return_df = pd.merge(first_purchase, first_return, on=['Client_id', 'SKU_id'], how='left')
sale_return_df['duration'] = (sale_return_df['return_date'] - sale_return_df['purchase_date']).dt.days
sale_return_df['event'] = sale_return_df['return_date'].notnull().astype(int)

kmf = KaplanMeierFitter()
kmf.fit(sale_return_df['duration'].fillna(0), event_observed=sale_return_df['event'])

kmf.plot_survival_function()
plt.title('Product Lifespan Survival Curve')
plt.xlabel('Days Since Purchase')
plt.ylabel('Survival Probability (Not Returned)')
plt.grid()
plt.show()


In [29]:
trans_df['is_return'] = (trans_df['Revenue'] < 0).astype(int)

client_returns = trans_df.groupby('Client_id').agg(
    has_return=('is_return', 'sum'),  
    total_purchases=('is_return', 'count') 
).reset_index()

client_returns['return_target'] = (client_returns['has_return'] > 0).astype(int)
client_df = client_df.merge(
    client_returns[['Client_id', 'return_target']], 
    on='Client_id', 
    how='left'
).fillna({'return_target': 0})

print("Original class distribution:")
print(client_df['return_target'].value_counts())

df_majority = client_df[client_df['return_target'] == 1]
df_minority = client_df[client_df['return_target'] == 0]
df_minority_upsampled = resample(df_minority,
                                 replace=True,
                                 n_samples=len(df_majority),
                                 random_state=42)

client_df_balanced = pd.concat([df_majority, df_minority_upsampled])

features = ['Age', 'Number_of_receipts', 'Total_revenue']
target = 'return_target'

client_df_balanced['Age'] = pd.to_numeric(client_df_balanced['Age'], errors='coerce')
client_df_balanced['Total_revenue'] = pd.to_numeric(client_df_balanced['Total_revenue'], errors='coerce')
client_df_balanced['Number_of_receipts'] = pd.to_numeric(client_df_balanced['Number_of_receipts'], errors='coerce')
client_df_balanced.dropna(subset=features + [target], inplace=True)

print("\nFinal class distribution:")
print(client_df_balanced[target].value_counts())

X = client_df_balanced[features]
y = client_df_balanced[target]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

model = LogisticRegression(max_iter=1000, class_weight='balanced')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Original class distribution:
return_target
0.0    377335
1.0     33796
Name: count, dtype: int64

Final class distribution:
return_target
1.0    17501
0.0    11136
Name: count, dtype: int64

Classification Report:
              precision    recall  f1-score   support

         0.0       0.49      0.74      0.59      2227
         1.0       0.76      0.51      0.61      3501

    accuracy                           0.60      5728
   macro avg       0.62      0.63      0.60      5728
weighted avg       0.65      0.60      0.60      5728


Confusion Matrix:
[[1656  571]
 [1715 1786]]


In [30]:
region_sales = trans_df.groupby('Region')['Revenue'].sum().reset_index()
region_sales = region_sales.sort_values(by='Revenue', ascending=False)

plt.figure(figsize=(12, 6))
sns.barplot(
    x='Region',
    y='Revenue',
    data=region_sales,
    palette='YlGnBu',
    hue='Region',        
    legend=False         
)

plt.title('Total Sales by Region')
plt.xlabel('Region')
plt.ylabel('Total Revenue')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()